In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
season_2014_15 = pd.read_csv('https://raw.githubusercontent.com/andrewkoo/aml_data/master/2014-15_season.csv')
season_2014_15['season'] = '2014-15'

season_2015_16 = pd.read_csv('https://raw.githubusercontent.com/andrewkoo/aml_data/master/2015-16_season.csv')
season_2015_16['season'] = '2015-16'

season_2016_17 = pd.read_csv('https://raw.githubusercontent.com/andrewkoo/aml_data/master/2016-17_season.csv')
season_2016_17['season'] = '2016-17'

season_2017_18 = pd.read_csv('https://raw.githubusercontent.com/andrewkoo/aml_data/master/2017-18_season.csv')
season_2017_18['season'] = '2017-18'

season_2018_19 = pd.read_csv('https://raw.githubusercontent.com/andrewkoo/aml_data/master/2018-19_season.csv')
season_2018_19['season'] = '2018-19'

season_2019_20 = pd.read_csv('https://raw.githubusercontent.com/andrewkoo/aml_data/master/2019-20_season.csv')
season_2019_20['season'] = '2019-20'

csv_data = pd.concat([season_2014_15, season_2015_16])
csv_data = pd.concat([csv_data, season_2016_17])
csv_data = pd.concat([csv_data, season_2017_18])
csv_data = pd.concat([csv_data, season_2018_19])
csv_data = pd.concat([csv_data, season_2019_20])

In [3]:
# split out 'Player' column to 'Name' and 'Player Key'
csv_data['Name'] = csv_data['Player'].apply(lambda x: x.split('\\')[0])
csv_data['Player Key'] = csv_data['Player'].apply(lambda x: x.split('\\')[1])
def playoffs(x):
    tmp = x.split(" ")
    return tmp[1] if len(tmp) > 1 else ''
csv_data['Playoffs'] = csv_data['Date'].apply(playoffs)
csv_data['Date'] = csv_data['Date'].apply(lambda x: x.split(' ')[0])

# integer representation of 'Date' column
def epoch_date(dt):
    pattern = '%Y-%m-%d'
    return int(time.mktime(time.strptime(dt, pattern)))
csv_data['epoch_date'] = csv_data['Date'].apply(epoch_date)

def draftkings_value(row):
    dk_value = 0
    dk_value += row['PTS']
    dk_value += row['3P'] * 0.5
    dk_value += row['TRB'] * 1.25
    dk_value += row['AST'] * 1.5
    dk_value += row['STL'] * 2
    dk_value += row['BLK'] * 2
    dk_value += row['TOV'] * -0.5

    # double double / triple double
    categories = ['PTS', 'TRB', 'AST', 'STL', 'BLK']
    check = (row[categories].values >= 10).sum()
    if check >= 3: 
        dk_value += 3
    elif check >= 2: 
        dk_value += 1.5
    return dk_value
csv_data['dk_value'] = csv_data.apply(draftkings_value, axis=1)

csv_data = csv_data[csv_data['Playoffs'] != '*']
csv_data = csv_data[csv_data['MP'] != 0]
csv_data = csv_data.sort_values('epoch_date')

csv_data.head()

,Rk,Player,Age,Pos,Tm,Unnamed: 5,Opp,Unnamed: 7,Date,GS,...,PF,PTS,GmSc,BPM,season,Name,Player Key,Playoffs,epoch_date,dk_value
27569,27570,Al-Farouq Aminu\aminual01,24-037,F,DAL,@,SAS,L,2014-10-28,0,...,1,0,-1.8,-32.3,2014-15,Al-Farouq Aminu,aminual01,,1414468800,2.00
27598,27599,Eric Gordon\gordoer01,25-307,G-F,NOP,NaN,ORL,W,2014-10-28,1,...,1,8,4.0,-5.1,2014-15,Eric Gordon,gordoer01,,1414468800,20.00
27597,27598,Ben Gordon\gordobe01,31-207,G,ORL,@,NOP,L,2014-10-28,0,...,3,5,0.1,-11.6,2014-15,Ben Gordon,gordobe01,,1414468800,11.50
27596,27597,Aaron Gordon\gordoaa01,19-042,F,ORL,@,NOP,L,2014-10-28,0,...,3,11,6.6,-1.2,2014-15,Aaron Gordon,gordoaa01,,1414468800,17.75
27595,27596,Manu Ginóbili\ginobma01,37-092,G,SAS,NaN,DAL,W,2014-10-28,0,...,4,20,15.1,6.0,2014-15,Manu Ginóbili,ginobma01,,1414468800,35.00


In [5]:
csv_data.to_csv('consolidated.csv', index=False)